# Linear Programming Assignment
## Lace Grant, Section 2

In [2]:
# Import Gurobi
import gurobipy as gp
from gurobipy import GRB

In [19]:
#### ORIGINAL MODEL ####

#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

#decision variables
x_1 = m.addVar(vtype=GRB.INTEGER, name= 'x_1', lb=0.0)
x_2 = m.addVar(vtype=GRB.INTEGER, name= 'x_2', lb=0.0)
x_3 = m.addVar(vtype=GRB.INTEGER, name= 'x_3', lb=0.0)
x_4 = m.addVar(vtype=GRB.INTEGER, name= 'x_4', lb=0.0)
x_5 = m.addVar(vtype=GRB.INTEGER, name= 'x_5', lb=0.0)

#set obj function
m.setObjective(2.75 * x_1 + 3.5 * x_2 + 3.25 * x_3 + 4 * x_4 + 4.25 * x_5 - 768)

#constraints
m.addConstr(x_1 + x_2 + x_3 + x_4 + x_5 <= 300, name = 'storage')
m.addConstr(4 * x_1 + 2 * x_4 + 3 * x_5 <= 512, name = 'turkey')
m.addConstr(4 * x_2 + 2 * x_4 + 3 * x_5 <= 576, name = 'beef')
m.addConstr(4 * x_3 + 2 * x_4 + 3 * x_5 <= 640, name = 'ham')
m.addConstr(x_1 + x_2 + 2 * x_3 + 2 * x_4 <= 512, name = 'cheese')
m.update()

#solve
m.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0xf165fa6b
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Found heuristic solution: objective 179.0000000
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 18 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 179.5000000

Root relaxation: objective 3.833333e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [20]:
#### SOLUTION AND ANALYSIS ####
#### NOTE: To calculate the shadow prices, I switched the model to continuous
#### It would not calculate my sensitivity analysis otherwise after hours of debugging :(

# get results & perform sensitivity analysis
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
    
    print("\nSensitivity Analysis:")
    for c in m.getConstrs():
        try:
            shadow_price = c.getAttr('Pi')
            slack_value = c.Slack
            if slack_value == 0:
                print(f"Constraint: {c.constrName}")
                print(f"  - Shadow Price: {shadow_price}")
                print(f"  - RHS: {c.RHS}")
                print(f"  - Slack: {slack_value}")
            else:
                print(f"Constraint: {c.constrName} is not binding.")
        except Exception as e:
            print(f"Error retrieving Pi for {c.constrName}: {e}")
else:
    print(f"Optimization did not succeed. Model status: {m.status}")

For optimal profit, produce the following amount of each sandwich:
x_1 = 10.0
x_2 = 26.0
x_3 = 41.0
x_4 = 197.0
x_5 = 26.0
Profit = 382.25

Sensitivity Analysis:
Error retrieving Pi for storage: Unable to retrieve attribute 'Pi'
Error retrieving Pi for turkey: Unable to retrieve attribute 'Pi'
Error retrieving Pi for beef: Unable to retrieve attribute 'Pi'
Error retrieving Pi for ham: Unable to retrieve attribute 'Pi'
Error retrieving Pi for cheese: Unable to retrieve attribute 'Pi'


In [21]:
#### CONTINUOUS MODEL FOR FINDING SHADOW PRICES 
#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

#decision variables
x_1 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_1', lb=0.0)
x_2 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_2', lb=0.0)
x_3 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_3', lb=0.0)
x_4 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_4', lb=0.0)
x_5 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_5', lb=0.0)

#set obj function
m.setObjective(2.75 * x_1 + 3.5 * x_2 + 3.25 * x_3 + 4 * x_4 + 4.25 * x_5 - 768)

#constraints
m.addConstr(x_1 + x_2 + x_3 + x_4 + x_5 <= 300, name = 'storage')
m.addConstr(4 * x_1 + 2 * x_4 + 3 * x_5 <= 512, name = 'turkey')
m.addConstr(4 * x_2 + 2 * x_4 + 3 * x_5 <= 576, name = 'beef')
m.addConstr(4 * x_3 + 2 * x_4 + 3 * x_5 <= 640, name = 'ham')
m.addConstr(x_1 + x_2 + 2 * x_3 + 2 * x_4 <= 512, name = 'cheese')
m.update()

#solve
m.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x219c73b7
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7750000e+31   1.475000e+31   1.775000e+01      0s
       5    3.8333333e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.833333333e+02


In [22]:
#### SOLUTION AND ANALYSIS ####
# get results & perform sensitivity analysis
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
    
    print("\nSensitivity Analysis:")
    for c in m.getConstrs():
        try:
            shadow_price = c.getAttr('Pi')
            slack_value = c.Slack
            if slack_value == 0:
                print(f"Constraint: {c.constrName}")
                print(f"  - Shadow Price: {shadow_price}")
                print(f"  - RHS: {c.RHS}")
                print(f"  - Slack: {slack_value}")
            else:
                print(f"Constraint: {c.constrName} is not binding.")
        except Exception as e:
            print(f"Error retrieving Pi for {c.constrName}: {e}")
else:
    print("Failed optimization.")

For optimal profit, produce the following amount of each sandwich:
x_1 = 9.333333333333334
x_2 = 25.33333333333333
x_3 = 41.33333333333333
x_4 = 197.33333333333334
x_5 = 26.666666666666668
Profit = 383.33333333333326

Sensitivity Analysis:
Constraint: storage
  - Shadow Price: 1.5
  - RHS: 300.0
  - Slack: 0.0
Constraint: turkey
  - Shadow Price: 0.22916666666666669
  - RHS: 512.0
  - Slack: 0.0
Constraint: beef
  - Shadow Price: 0.41666666666666663
  - RHS: 576.0
  - Slack: 0.0
Constraint: ham
  - Shadow Price: 0.27083333333333337
  - RHS: 640.0
  - Slack: 0.0
Constraint: cheese
  - Shadow Price: 0.3333333333333333
  - RHS: 512.0
  - Slack: 0.0


In [26]:
#### FINDING RHS RANGE ####

for c in m.getConstrs():
        shadow_price = c.getAttr('Pi')
        rhs_value = c.RHS
        slack_value = c.Slack
        if slack_value == 0:
            range_upper = rhs_value + (1 / abs(shadow_price))
            range_lower = rhs_value - (1 / abs(shadow_price))
            print(f"Constraint: {c.constrName}")
            print(f"  - Shadow Price: {shadow_price}")
            print(f"  - RHS: {rhs_value}")
            print(f"  - Slack: {slack_value}")
            print(f"  - Upper RHS bound: {range_upper}")
            print(f"  - Lower RHS bound: {range_lower}")
        else:
            print("No RHS range.")

Constraint: storage
  - Shadow Price: 1.5
  - RHS: 300.0
  - Slack: 0.0
  - Upper RHS bound: 300.6666666666667
  - Lower RHS bound: 299.3333333333333
Constraint: turkey
  - Shadow Price: 0.22916666666666669
  - RHS: 512.0
  - Slack: 0.0
  - Upper RHS bound: 516.3636363636364
  - Lower RHS bound: 507.6363636363636
Constraint: beef
  - Shadow Price: 0.41666666666666663
  - RHS: 576.0
  - Slack: 0.0
  - Upper RHS bound: 578.4
  - Lower RHS bound: 573.6
Constraint: ham
  - Shadow Price: 0.27083333333333337
  - RHS: 640.0
  - Slack: 0.0
  - Upper RHS bound: 643.6923076923077
  - Lower RHS bound: 636.3076923076923
Constraint: cheese
  - Shadow Price: 0.3333333333333333
  - RHS: 512.0
  - Slack: 0.0
  - Upper RHS bound: 515.0
  - Lower RHS bound: 509.0


In [32]:
#### INTEGER MODEL - 5 DOLLAR CLUB PRICE
#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

#decision variables
x_1 = m.addVar(vtype=GRB.INTEGER, name= 'x_1', lb=0.0)
x_2 = m.addVar(vtype=GRB.INTEGER, name= 'x_2', lb=0.0)
x_3 = m.addVar(vtype=GRB.INTEGER, name= 'x_3', lb=0.0)
x_4 = m.addVar(vtype=GRB.INTEGER, name= 'x_4', lb=0.0)
x_5 = m.addVar(vtype=GRB.INTEGER, name= 'x_5', lb=0.0)

#set obj function
m.setObjective(2.75 * x_1 + 3.5 * x_2 + 3.25 * x_3 + 5 * x_4 + 4.25 * x_5 - 768)

#constraints
m.addConstr(x_1 + x_2 + x_3 + x_4 + x_5 <= 300, name = 'storage')
m.addConstr(4 * x_1 + 2 * x_4 + 3 * x_5 <= 512, name = 'turkey')
m.addConstr(4 * x_2 + 2 * x_4 + 3 * x_5 <= 576, name = 'beef')
m.addConstr(4 * x_3 + 2 * x_4 + 3 * x_5 <= 640, name = 'ham')
m.addConstr(x_1 + x_2 + 2 * x_3 + 2 * x_4 <= 512, name = 'cheese')
m.update()

#solve
m.optimize()

#results
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
else: print('No solution found.')

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x0c5a24ff
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Found heuristic solution: objective 179.0000000
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 18 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 179.5000000

Root relaxation: objective 5.853333e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [34]:
#### CONTINUOUS MODEL FOR FINDING SHADOW PRICES - 5 DOLLAR CLUB PRICE
#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

#decision variables
x_1 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_1', lb=0.0)
x_2 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_2', lb=0.0)
x_3 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_3', lb=0.0)
x_4 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_4', lb=0.0)
x_5 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_5', lb=0.0)

#set obj function
m.setObjective(2.75 * x_1 + 3.5 * x_2 + 3.25 * x_3 + 5 * x_4 + 4.25 * x_5 - 768)

#constraints
m.addConstr(x_1 + x_2 + x_3 + x_4 + x_5 <= 300, name = 'storage')
m.addConstr(4 * x_1 + 2 * x_4 + 3 * x_5 <= 512, name = 'turkey')
m.addConstr(4 * x_2 + 2 * x_4 + 3 * x_5 <= 576, name = 'beef')
m.addConstr(4 * x_3 + 2 * x_4 + 3 * x_5 <= 640, name = 'ham')
m.addConstr(x_1 + x_2 + 2 * x_3 + 2 * x_4 <= 512, name = 'cheese')
m.update()

#solve
m.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x3c909e4c
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8750000e+31   1.475000e+31   1.875000e+01      0s
       5    5.8533333e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.853333333e+02


In [35]:
#### SOLUTION AND ANALYSIS ####
# get results & perform sensitivity analysis
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
    
    print("\nSensitivity Analysis:")
    for c in m.getConstrs():
        try:
            shadow_price = c.getAttr('Pi')
            slack_value = c.Slack
            if slack_value == 0:
                print(f"Constraint: {c.constrName}")
                print(f"  - Shadow Price: {shadow_price}")
                print(f"  - RHS: {c.RHS}")
                print(f"  - Slack: {slack_value}")
            else:
                print(f"Constraint: {c.constrName} is not binding.")
        except Exception as e:
            print(f"Error retrieving Pi for {c.constrName}: {e}")
else:
    print("Failed optimization.")

For optimal profit, produce the following amount of each sandwich:
x_1 = 0.0
x_2 = 16.0
x_3 = 32.0
x_4 = 216.0
x_5 = 26.666666666666668
Profit = 585.3333333333333

Sensitivity Analysis:
Constraint: storage is not binding.
Constraint: turkey
  - Shadow Price: 0.5416666666666666
  - RHS: 512.0
  - Slack: 0.0
Constraint: beef
  - Shadow Price: 0.6041666666666666
  - RHS: 576.0
  - Slack: 0.0
Constraint: ham
  - Shadow Price: 0.2708333333333333
  - RHS: 640.0
  - Slack: 0.0
Constraint: cheese
  - Shadow Price: 1.0833333333333335
  - RHS: 512.0
  - Slack: 0.0


In [37]:
#### INTEGER MODEL - MEAT REDUCTION
#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

#decision variables
x_1 = m.addVar(vtype=GRB.INTEGER, name= 'x_1', lb=0.0)
x_2 = m.addVar(vtype=GRB.INTEGER, name= 'x_2', lb=0.0)
x_3 = m.addVar(vtype=GRB.INTEGER, name= 'x_3', lb=0.0)
x_4 = m.addVar(vtype=GRB.INTEGER, name= 'x_4', lb=0.0)
x_5 = m.addVar(vtype=GRB.INTEGER, name= 'x_5', lb=0.0)

#set obj function
m.setObjective(2.75 * x_1 + 3.5 * x_2 + 3.25 * x_3 + 4 * x_4 + 4.25 * x_5 - 768)

#constraints
m.addConstr(x_1 + x_2 + x_3 + x_4 + x_5 <= 300, name = 'storage')
m.addConstr(4 * x_1 + 2 * x_4 + 2.5 * x_5 <= 512, name = 'turkey')
m.addConstr(4 * x_2 + 2 * x_4 + 2.5 * x_5 <= 576, name = 'beef')
m.addConstr(4 * x_3 + 2 * x_4 + 2.5 * x_5 <= 640, name = 'ham')
m.addConstr(x_1 + x_2 + 2 * x_3 + 2 * x_4 <= 512, name = 'cheese')
m.update()

#solve
m.optimize()

#results
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
else: print('No solution found.')

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x20222840
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Found heuristic solution: objective 179.0000000
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 18 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 179.5000000

Root relaxation: objective 3.980000e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [38]:
#### CONTINUOUS MODEL - MEAT REDUCTION
#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

#decision variables
x_1 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_1', lb=0.0)
x_2 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_2', lb=0.0)
x_3 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_3', lb=0.0)
x_4 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_4', lb=0.0)
x_5 = m.addVar(vtype=GRB.CONTINUOUS, name= 'x_5', lb=0.0)

#set obj function
m.setObjective(2.75 * x_1 + 3.5 * x_2 + 3.25 * x_3 + 4 * x_4 + 4.25 * x_5 - 768)

#constraints
m.addConstr(x_1 + x_2 + x_3 + x_4 + x_5 <= 300, name = 'storage')
m.addConstr(4 * x_1 + 2 * x_4 + 2.5 * x_5 <= 512, name = 'turkey')
m.addConstr(4 * x_2 + 2 * x_4 + 2.5 * x_5 <= 576, name = 'beef')
m.addConstr(4 * x_3 + 2 * x_4 + 2.5 * x_5 <= 640, name = 'ham')
m.addConstr(x_1 + x_2 + 2 * x_3 + 2 * x_4 <= 512, name = 'cheese')
m.update()

#solve
m.optimize()

#results
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
else: print('No solution found.')

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x5058a18d
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7750000e+31   1.437500e+31   1.775000e+01      0s
       5    3.9800000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.980000000e+02
For optimal profit, produce the following amount of each sandwich:
x_1 = 4.0
x_2 = 20.0
x_3 = 36.0
x_4 = 208.0
x_5 = 32.0


In [39]:
#### SOLUTION AND ANALYSIS ####
# get results & perform sensitivity analysis
if m.status == GRB.OPTIMAL:
    print('For optimal profit, produce the following amount of each sandwich:')
    for v in m.getVars():
        print(f'{v.VarName} = {v.X}')
    print(f'Profit = {m.objVal}')
    
    print("\nSensitivity Analysis:")
    for c in m.getConstrs():
        try:
            shadow_price = c.getAttr('Pi')
            slack_value = c.Slack
            if slack_value == 0:
                print(f"Constraint: {c.constrName}")
                print(f"  - Shadow Price: {shadow_price}")
                print(f"  - RHS: {c.RHS}")
                print(f"  - Slack: {slack_value}")
            else:
                print(f"Constraint: {c.constrName} is not binding.")
        except Exception as e:
            print(f"Error retrieving Pi for {c.constrName}: {e}")
else:
    print("Failed optimization.")

For optimal profit, produce the following amount of each sandwich:
x_1 = 4.0
x_2 = 20.0
x_3 = 36.0
x_4 = 208.0
x_5 = 32.0
Profit = 398.0

Sensitivity Analysis:
Constraint: storage
  - Shadow Price: 1.5
  - RHS: 300.0
  - Slack: 0.0
Constraint: turkey
  - Shadow Price: 0.275
  - RHS: 512.0
  - Slack: 0.0
Constraint: beef
  - Shadow Price: 0.4625
  - RHS: 576.0
  - Slack: 0.0
Constraint: ham
  - Shadow Price: 0.36250000000000004
  - RHS: 640.0
  - Slack: 0.0
Constraint: cheese
  - Shadow Price: 0.15
  - RHS: 512.0
  - Slack: 0.0


In [40]:
#### THREE SANDWICH MODEL ####

#create model object
m = gp.Model('deli')

#specific obj function and time limit
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

# Decision variables (Integer type, limited to 150 units)
x_3 = m.addVar(vtype=GRB.INTEGER, name='x_3', lb=0.0, ub=150)  # Sandwich 3 (units made)
x_4 = m.addVar(vtype=GRB.INTEGER, name='x_4', lb=0.0, ub=150)  # Sandwich 4 (units made)
x_5 = m.addVar(vtype=GRB.INTEGER, name='x_5', lb=0.0, ub=150)  # Sandwich 5 (units made)

#set obj function
profit_3 = 3.25
profit_4 = 4.0
profit_5 = 4.25
m.setObjective(profit_3 * x_3 + profit_4 * x_4 + profit_5 * x_5, GRB.MAXIMIZE)

# Constraints based on the ingredients used in each sandwich
m.addConstr(4 * x_3 + 2 * x_4 + 2.5 * x_5 <= 512, name='turkey')
m.addConstr(4 * x_3 + 2 * x_4 + 2.5 * x_5 <= 576, name='beef')
m.addConstr(4 * x_3 + 2 * x_4 + 2.5 * x_5 <= 640, name='ham')

# Storage constraint: Total sandwiches to be made cannot exceed 300 (unchanged)
m.addConstr(x_3 + x_4 + x_5 <= 300, name='storage')

# Solve the model
m.optimize()

# Output the results
if m.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    for v in m.getVars():
        print(f'{v.varName} = {v.x}')
    print(f'Profit = {m.objVal}')
else:
    print(f'Optimization did not succeed. Model status: {m.status}')


Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 3 columns and 12 nonzeros
Model fingerprint: 0xb3d189c7
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [2e+02, 2e+02]
  RHS range        [3e+02, 6e+02]
Found heuristic solution: objective 416.0000000
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 417.7500000

Root relaxation: objective 9.604000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 